# Etudiantes
Lina Berroug
et
Jessica Karega

In [2]:
#IMPORTATION DE MODULES

import json
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [13]:
# Télécharger les stopwords en français si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/jesse/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jesse/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jesse/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Introduction
Ce notebook analyse les tweets d'un fichier JSON pour extraire des informations et les exploiter grâce à un dataFrame

Nous utiliserons les bibliothèques suivantes :
- `json` : pour manipuler les fichiers JSON
- `pandas` : pour analyser les données tabulaires
- `re` : pour extraire ou supprimer des expressions régulière d'un tweet
- `TextBlob` : pour l'analyse de sentiments
- `matplotlib` : pour visualiser sous forme de graphiques les données


### Diagramme des étapes de traitement
Schéma qui montre les étapes suivies pour le projet :

<img src="diagramme.png" alt="Description de l'image" style="width:40%; height:auto;">




In [16]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE

#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.contenu = contenu

    def Nettoyage_Tweet(self):
        """Fonction qui supprime les caractères spéciaux d'un tweet"""
        enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self.contenu)
        return enlever_speciaux

class ZA:
    def __init__(self, id, auteur, texte): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.id = id
        self.auteur = auteur
        self.text = texte
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()

    
    def Extraction_Sentiments(self): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(self.text)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
    def Extraction_Hashtags(self):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', self.text)
        return hashtags
    
    def Extraction_Mentions(self):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', self.text)

        return mentions


    def Id_Topic(tweet):

        def traitement_texte(text):
            text= text.lower() # mettre en minuscule
            text= re.sub(r'[^a-z\s]', '', text) 
            return text
        def get_stopwords():
            return stopwords.words('french')
        stopwords_list = get_stopwords()
        tweets= []
        with open ('ZA.json','r',encoding='utf-8') as f:
            donnee= json.load(f)
        
        tweets = [infos["tweet"] for infos in donnee]
        
        texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
        vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
        a = vecteur.fit_transform(texte_traite)

        # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
        lda = LatentDirichletAllocation(n_components=2, random_state=42)
        lda.fit(a) # on utilise LDA sur la matrice X 

        topics = []
        # Récupération du vocabulaire extrait par TfidfVectorizer
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.

            topics.append(f": {', '.join(top_words)}")

        tweet_traite = traitement_texte(tweet)
        vecteur = TfidfVectorizer(stop_words= stopwords_list) # Utilise les stopwords en français intégrés dans sklearn
        vecteur_tweet= vecteur.transform([tweet_traite]) 
        probabilities = lda.transform(vecteur_tweet)
        topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

        print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")


def Zone_Atterisssage (Fichier): # incomplet
        """Stockage du tweet dans le fichier json zone d'atterissage"""
        ZA_liste=[]
        
        with open (Fichier,'r',encoding='utf-8') as f:
            donnee= json.load(f)
            
        for infos_tweets in donnee:
            id = infos_tweets.get("id")
            auteur = infos_tweets.get("author_id")
            contenu = infos_tweets.get("text")

            tweet = Tweet(contenu)
            contenu_nettoye = tweet.Nettoyage_Tweet()
    

            tweet_analyse = ZA(id, auteur, contenu_nettoye)


            dict_tweet = {
                "id": tweet_analyse.id,
                "auteur": tweet_analyse.auteur,
                "tweet": contenu_nettoye,
                "texte": tweet_analyse.text,
                "hashtags" : tweet_analyse.hashtags,
                "mentions" : tweet_analyse.mentions,
                "sentiment" : tweet_analyse.sentiment
            }


            ZA_liste.append(dict_tweet)

        
        
        with open('ZA.json','w', encoding='utf-8') as f:
            json.dump(ZA_liste, f, indent=4, ensure_ascii=False)


# --> il faut charger ces données dans une date frame : df = pd.DataFrame(tweets)





In [ ]:
#DATAFRAME
def CreerDataFrame(fichier_json):
    df  = pd.read_json(fichier_json)
    return df

#Analyses avancées 

def Top_K_element(df, colonne, k=3):
    """
    Retourne les K éléments les plus fréquents dans une colonne donnée.
    """
    top_k = df[colonne].explode().value_counts().head(k) #a méthode .explode() transforme chaque élément d'une liste dans une ligne distincte.
    element= top_k.index.tolist()  # Liste des hashtags
    counts = top_k.values.tolist()
    return element,counts

def publications_par_hashtag(df):
    """
    Retourne le nombre de publications par hashtag.
    """
    hashtags_explodes = df['hashtags'].explode()
    hashtag_counts = hashtags_explodes.value_counts()

    hashtags = hashtag_counts.index.tolist()  # Liste des hashtags
    counts = hashtag_counts.values.tolist()   # Liste des nombres d'occurrences
    return hashtags, counts

def publications_par_utilisateurs(df):
    """
    Retourne le nombre de publications par utilisateur.
    """
    utilisateur_counts = df['auteur'].value_counts()
    utilisateur = utilisateur_counts.index.tolist()  # Liste des utilisateurs
    counts = utilisateur_counts.values.tolist()   # Liste des nombres d'occurrences
    return utilisateur, counts

def publications_par_topic(df):
    """
    Retourne le nombre de publications par topics.
    """
    topics_counts = df['topics'].value_counts()
    topic = topics_counts.index.tolist()  # Liste des topics
    counts = topics_counts.values.tolist()   # Liste des nombres d'occurrences
    return topic , counts

def tweets_par_utilisateur(df,utilisateur):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['auteur'] == utilisateur]
    tweets = filtre['contenu'].tolist() 
    utilisateurs = filtre ['auteur'].tolist() 
    return utilisateurs,tweets
    
def ensemble_de_tweets_mentionnant_utilisateur(df,utilisateur_mentionné):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['mentions'] == utilisateur_mentionné]
    return filtre

def ensemble_de_utilisateurs_mentions_hashtag(df, hashtag):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['hashtag'] == hashtag]
    return filtre
    
def ensemble_de_utilisateurs_mentionnés_utilisateur(df, utilisateur):
    """
    Retourne les mentions d'un utilisateur.
    """
    tweets_u = df[df['auteur'] == utilisateur]
    mentions = tweets_u['mentions'].explode()
    return list(mentions)




hashtags
#CIV                2
#twitter225         1
#SupportriceMazo    1
#domie              1
#jifa               1
#versailles         1
#nocturne           1
#appollon           1
Name: count, dtype: int64

--> Pour améliorer ce code il faudrait utiliser nltk pour la tokenisation et la lemmenisation : ça va couper les tweet en petit mots et reduire les mots en leur forme conique ( je n'arrive pas à utilser nltk pour faire ça, erreur)

Ducoup la c pas précis

In [ ]:
# Visualisation des résultats


def nbr_hashtag(df):
    hashtags, counts = publications_par_hashtag(df)
    plt.bar(hashtags,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel("Hashtags")
    plt.ylabel("Fréquence")
    plt.show()

def nbr_utilisateur(df):
    utilisateur, counts = publications_par_utilisateurs(df)
    plt.bar(utilisateur,counts, color = 'rouge')
    plt.title("Nombre de publication par utilisateur")
    plt.xlabel("utilisateurs")
    plt.ylabel("Fréquence")
    plt.show()

def nbr_topic(df):
    topic, counts = publications_par_topic(df)
    plt.bar(topic,counts, color = 'rouge')
    plt.title("Nombre de publication par topic")
    plt.xlabel("topics")
    plt.ylabel("Fréquence")
    plt.show()

def top_hashtgs(df):
    element,counts = Top_K_element(df, 'hashtags', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel(f"top {k} hashtags")
    plt.ylabel("Fréquence")
    plt.show()

def top_topic(df):
    element,counts = Top_K_element(df, 'topic', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel('topics')
    plt.ylabel("Fréquence")
    plt.show()

def top_auteur(df):
    element,counts = Top_K_element(df, 'auteur', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Top k utilisateur")
    plt.xlabel('Utilisateur')
    plt.ylabel("Fréquence")
    plt.show()

def top_hashtgs(df):
    element,counts = Top_K_element(df, 'mentions', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel('mentions')
    plt.ylabel("Fréquence")
    plt.show()


    

In [ ]:
t= "@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part , JO prochain on ira en demi final au moins. BRAVO à vous . #SupportriceMazo #domie #CIV"


def Nettoyage_Tweet(tweet):
    """Fonction qui supprime les caractères spéciaux d'un tweet"""
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', tweet)
    return enlever_speciaux

def Extraction_Sentiments(tweet): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(tweet)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
def Extraction_Hashtags(tweet):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
def Extraction_Mentions(tweet):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', tweet)
        return mentions


print(Nettoyage_Tweet(t))
print(Extraction_Sentiments(t))
print(Extraction_Hashtags(t))
print(Extraction_Mentions(t))




@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part  JO prochain on ira en demi final au moins BRAVO  vous  #SupportriceMazo #domie #CIV
neutre
['#SupportriceMazo', '#domie', '#CIV']
['@ericbailly24', '@maxigr04del']


In [ ]:
donne= [
    {"tweet": "Le réchauffement climatique menace la biodiversité."},
    {"tweet": "La pollution des océans est un problème mondial."},
    {"tweet": "Les voitures électriques sont une solution écologique."},
    {"tweet": "Les panneaux solaires deviennent plus efficaces chaque année."},
    {"tweet": "La déforestation accélère le changement climatique."}
]


def Id_Topic(tweet):

    def traitement_texte(text):
        text= text.lower() # mettre en minuscule
        text= re.sub(r'[^a-z\s]', '', text) 
        return text
    def get_stopwords():
        return stopwords.words('french')
    stopwords_list = get_stopwords()
    tweets= []
    
        
    tweets = [infos["tweet"] for infos in donne]
        
    texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
    vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
    a = vecteur.fit_transform(texte_traite)

    # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
    lda = LatentDirichletAllocation(n_components=2, random_state=42)
    lda.fit(a) # on utilise LDA sur la matrice X 

    topics = []
    # Récupération du vocabulaire extrait par TfidfVectorizer
    words = vecteur.get_feature_names_out()
    for i, topic_weights in enumerate(lda.components_):
        top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
        top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.

        topics.append(f": {', '.join(top_words)}")

    tweet_traite = traitement_texte(tweet)
    vecteur_tweet= vecteur.transform([tweet_traite]) 
    probabilities = lda.transform(vecteur_tweet)
    topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

    print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")

tweet = "Le réchauffement climatique menace la biodiversité."
Id_Topic(tweet)


Le tweet :
Le réchauffement climatique menace la biodiversité.
Appartient au Topic : biodiversit, rchauffement
